In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


path = Path(base_dir + 'data/bears')

dest = path/folder

dest.mkdir(parents=True, exist_ok=True)

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.text import *
from fastai import *
import json
import html
import re
import pickle
import random
import pandas as pd
import numpy as np
from pathlib import Path
import sklearn
from sklearn import model_selection
from functools import partial
from collections import Counter, defaultdict
from pandas.io.json import json_normalize
import numpy as np
import torch
import torch.nn as nn
import torch.utils 
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import dataset, dataloader
import torch.optim as optim
import torch.nn.functional as F
from nltk.tokenize import  sent_tokenize
import time
import math
import sys
import data
import joblib
from random import shuffle
from itertools import permutations, combinations

#path = Path(base_dir + 'data/PAN14')
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/'
#path = Path(base_dir + '/PAN14/')



token_files = base_dir+'PAN14/tokens/'
model_files = base_dir+'PAN14/models/'
bs=64
TRAINDATAPATH = base_dir+"PAN14/pan14_train_english-essays/"
TESTDATAPATH = base_dir+"PAN14/pan14_test02_english-essays/"
FNAMES = ['known01','known02','known03','known04','known05', 'unknown']
KCOLS=['known01','known02','known03','known04','known05']
LABELCOL="answer"
UNKOWN="unknown"
np.random.seed=42
BOD = 'x_bod' # beginning-of-doc tag

re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

def read_dataset(path):
    ds=pd.read_json(path+'/truth.json')
    ds=json_normalize(ds['problems'])
    ds['known01']=None
    ds['known02']=None
    ds['known03']=None
    ds['known04']=None
    ds['known05']=None
    ds['unknown']=None
    ds.set_index('name', drop=True, inplace=True)
    ds=ds[['known01','known02','known03','known04','known05', 'unknown', 'answer']]
    dirs = []
    docs = []

    for i, x in enumerate(os.walk(path)):
        if i:
            for fname in x[2]:
                with open(path+dirs[i-1]+'/'+fname, 'r') as f:
                    text = f.read().strip()
                    doc = ' '.join(sent_tokenize(text)).strip()
                    ds.loc[dirs[i-1],fname[:-4]]=doc
        else:
            dirs = x[1]

    return ds

def match_unknowns(path):
    ds=pd.read_json(path+'/truth.json')
    ds=json_normalize(ds['problems'])
    ds['known01']=None
    ds['known02']=None
    ds['known03']=None
    ds['known04']=None
    ds['known05']=None
    ds['unknown']=None
    ds.set_index('name', drop=True, inplace=True)
    ds=ds[['known01','known02','known03','known04','known05', 'unknown', 'answer']]
    dirs = []
    docs = []

    ds = read_dataset(path)
            
    grouped=ds.groupby(['unknown'])
    dupes=[]
    for utext, group in grouped:
        if len(group.index) > 1:
            dupes.append(group)


    newrows=pd.DataFrame(columns=['known01','known02','known03','known04','known05', 'unknown'])
    for dupe in dupes:
        dupe.reset_index(drop=True, inplace=True)
        yes=dupe.loc[dupe.answer == "Y"]
        yes.reset_index(drop=True, inplace=True)
        no=dupe.loc[dupe.answer == "N"]
        no.reset_index(drop=True, inplace=True)
        for col in ['known01','known02','known03','known04','known05']:
            if no[col] is not None:
                newrows=newrows.append(pd.DataFrame(data={'known01':yes.known01,'known02':yes.known02,
                                                          'known03':yes.known03, 'known04':yes.known04,
                                                          'known05':yes.known05,'unknown':no[col], 
                                                          'answer':'N'}), sort=False)
    newrows=newrows.dropna(subset=['unknown'])
    df = pd.concat([ds, newrows])

    return df


def shuffle_text(text, n=4):
    pars = [text[i:i+int(len(text)/n)] for i in range(0, len(text), int(len(text)/n))]
    shuffle(pars)
    return ''.join(pars)


def load_doc_pairs(df, augment=False):
   
    s0s = []
    s1s = []
    labels = [1 if label == 'Y' else 0 for label in df[LABELCOL].tolist()]
    
    y=[]
    unknowns = df[UNKOWN].tolist()
    for i, label in enumerate(labels):
        for col in KCOLS:
            knowns = df[col].tolist()

            s0 = knowns[i]
            if s0 is not None:
                s1 = unknowns[i]
                s0s.append(s0)
                s1s.append(s1)
                y.append(label)
                
                if augment:
                    s0inflated=shuffle_text(s0)
                    s1inflated=shuffle_text(s1)
                    s0s.append(s0inflated)
                    s1s.append(s1inflated)
                    y.append(label)
    data = pd.DataFrame(data={"known":s0s, "unknown":s1s, "label":y}) 
    
    return data





In [8]:

df_train = read_dataset(TRAINDATAPATH)
df_test = read_dataset(TESTDATAPATH)

doc_pairs_train = load_doc_pairs(df_train, False)
doc_pairs_test = load_doc_pairs(df_test, False)

joblib.dump(doc_pairs_train, f'{model_files}traindf-b.pkl')
joblib.dump(doc_pairs_test, f'{model_files}testdf-b.pkl')
doc_pairs_train['is_valid'] = False
doc_pairs_test['is_valid'] = True
df = pd.concat([doc_pairs_train,doc_pairs_test]).sample(frac=1)
joblib.dump(df, 'data.pkl')

ValueError: ignored